# Images/Labels/Tables

In this notebook we will show how to use the `Image`, `Label` and `Table` objects to do image processing.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt

from ngio import open_omezarr_container
from ngio.utils import download_ome_zarr_dataset

# Download the dataset
download_dir = Path(".").absolute().parent.parent / "data"
hcs_path = download_ome_zarr_dataset("CardiomyocyteSmallMip", download_dir=download_dir)
image_path = hcs_path / "B" / "03" / "0"

# Open the ome-zarr container
omezarr = open_omezarr_container(image_path)

## Images

Images can be loaded from a `OmeZarrContainer` object.

In [ ]:
image = omezarr.get_image()

print("Image information:")
print(f"{image.shape=}")
print(f"{image.pixel_size=}")
print(f"{image.channel_labels=}")
print(f"{image.dimensions=}")

## Data Loading

The `Image` object created is a lazy object, meaning that the image is not loaded into memory until it is needed.
To get the image data from disk we can use the `.array` attribute or we can get it as a `dask.array` object using the `.dask_array` attribute.

In [ ]:
image.get_array(mode="dask")  # this call is lazy

Images can be queried for any axes, in any order

In [ ]:
print("On disk shape: ", image.shape)

# Axes order can be specified
# if an axis is not present in the array, it will be added as a singleton dimension
array = image.get_array(axes_order=["x", "t", "c", "y", "z"], mode="dask")

print("Array shape: ", array.shape)

## RoiTable/Image Interaction

`roi` objects from a `roi_table` can be used to extract a region of interest from an image or a label.

In [ ]:
roi_table = omezarr.get_table("FOV_ROI_table", check_type="roi_table")
# Get a roi by name
roi = roi_table.get("FOV_1")
print(f"{roi=}")

# .get_roi works exactly like .get_array
# the only difference is that x, y, z, axes are queried from the roi object
image_roi_1 = image.get_roi(roi=roi, c=0, mode="dask")
image_roi_1

The roi object can is defined in physical coordinates, and can be used to extract the region of interest from the image or label at any resolution.

In [ ]:
image_2 = omezarr.get_image(path="2")
# Two images at different resolutions
print(f"{image.pixel_size=}")
print(f"{image_2.pixel_size=}")

# Get roi for higher resolution image
image_1_roi_1 = image.get_roi(roi=roi, c=0, mode="dask")

# Get roi for lower resolution image
image_2_roi_1 = image_2.get_roi(roi=roi, c=0, mode="dask")

# Plot the two images side by side
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(image_1_roi_1[0, 0], cmap="gray")
axs[1].imshow(image_2_roi_1[0, 0], cmap="gray")
plt.show()

# Writing Images

Similarly to the `.array()`  we can use the `.set_array()` (or `set_array_from_roi`) method to write part of an image to disk.

In [ ]:
import numpy as np

# Get a small slice of the image
small_slice = image.get_array(x=slice(1000, 2000), y=slice(1000, 2000))

# Set the sample slice to zeros
zeros_slice = np.zeros_like(small_slice)
image.set_array(patch=zeros_slice, x=slice(1000, 2000), y=slice(1000, 2000))


# Load the image from disk and show the edited image
nuclei = omezarr.get_label("nuclei", path="0")
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

axs[0].imshow(image.get_array()[0, 0], cmap="gray")
axs[1].imshow(nuclei.get_array()[0])
for ax in axs:
    ax.axis("off")
plt.tight_layout()
plt.show()

# Add back the original slice to the image
image.set_array(patch=small_slice, x=slice(1000, 2000), y=slice(1000, 2000))

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(image.get_array()[0, 0], cmap="gray")
axs[1].imshow(nuclei.get_array()[0])
for ax in axs:
    ax.axis("off")
plt.tight_layout()
plt.show()

## Deriving a new label

When doing image analysis, we often need to create new labels or tables. The `ngff_image` allows us to simply create new labels and tables.

In [ ]:
# Create a a new label object and set it to a simple segmentation
new_label = omezarr.derive_label("new_label", overwrite=True)

simple_segmentation = image.get_array(c=0) > 100
simple_segmentation = simple_segmentation[0]
new_label.set_array(simple_segmentation)

# make a subplot with two image show side by side
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(image.get_array()[0, 0], cmap="gray")
axs[1].imshow(new_label.get_array()[0])
for ax in axs:
    ax.axis("off")
plt.tight_layout()
plt.show()

### Image Consolidation

Every time we modify a label or a image, we are modifying the on-disk data on one layer only. 
This means that if we have the image saved in multiple resolutions, we need to consolidate the changes to all resolutions.
To do so, we can use the `.consolidate()` method.

In [ ]:
label_0 = omezarr.get_label("new_label", path="0")
label_2 = omezarr.get_label("new_label", path="2")

label_before_consolidation = label_2.zarr_array[...]

# Consolidate the label
label_0.consolidate()

label_after_consolidation = label_2.zarr_array[...]


# make a subplot with two image show side by side
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(label_before_consolidation[0], cmap="gray")
axs[1].imshow(label_after_consolidation[0], cmap="gray")
for ax in axs:
    ax.axis("off")
plt.tight_layout()
plt.show()

## Creating a new table

We can simply create a new table by create a new `Table` object from a pandas dataframe.
For a simple feature table the only reuiremnt is to have a integer column named `label` that will be used to link the table to the objects in the image.


In [ ]:
import numpy as np
import pandas as pd

from ngio.tables import FeatureTable

print(f"List of all tables: {omezarr.list_tables()}")


nuclei = omezarr.get_label("nuclei", path="0")
roi_table = omezarr.get_table("FOV_ROI_table", check_type="roi_table")

# Create a table with random features for each nuclei in each ROI
list_of_records = []
for roi in roi_table.rois():
    nuclei_in_roi = nuclei.get_roi(roi, mode="numpy")
    for nuclei_id in np.unique(nuclei_in_roi)[1:]:
        list_of_records.append(
            {
                "label": nuclei_id,
                "feat1": np.random.rand(),
                "feat2": np.random.rand(),
                "ROI": roi.name,
            }
        )

feat_df = pd.DataFrame.from_records(list_of_records)

feat_table = FeatureTable(feat_df, reference_label="nuclei")

omezarr.add_table("new_feature_table", feat_table, overwrite=True)

In [ ]:
feat_table = omezarr.get_table("new_feature_table")
feat_table.dataframe